# Imports

In [14]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import folium
from ipywidgets import widgets
from IPython.display import display, clear_output
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.widgets import GraphWidget
import plotly
plotly.tools.set_credentials_file(username='silkeh', api_key='GwBAmaelgKfgkrmuL691')

separator = '\t'
#Eva
filepath = 'D:/Workspace/_DataMining/DataSets'
#Silke
#filepath = 'C:/Users/Silke/Documents/GitHub/DataMiningAP'

#output filepath for new csv files
#Eva
#outputFilepath = 'D:/Workspace/_DataMining/DataSets/IMDB_new'
#Silke
#outputFilepath = 'C:/Users/Silke/Documents/GitHub/DataMiningAP/IMDB_new'

# IMDB
## Creation of RDD's and DF's

In [2]:
#TITLE.BASICS
#tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres
titleBasics_data = sc.textFile(filepath + '/IMDB/titleBasics.tsv')
titleBasics_rdd = titleBasics_data.filter(lambda l: 'primaryTitle' not in l).map(lambda l: l.split(separator))

#create schema for df
tbFields = []
tbFields.append(StructField('tconst', StringType(), True))
tbFields.append(StructField('titleType', StringType(), True))
tbFields.append(StructField('primaryTitle', StringType(), True))
tbFields.append(StructField('originalTitle', StringType(), True))
tbFields.append(StructField('isAdult', StringType(), True))
tbFields.append(StructField('startYear', StringType(), True))
tbFields.append(StructField('endYear', StringType(), True))
tbFields.append(StructField('runtimeMinutes', StringType(), True))
tbFields.append(StructField('genres', StringType(), True))                           

tbSchema = StructType(tbFields)

#create df
titleBasics_df = sqlContext.createDataFrame(titleBasics_rdd, tbSchema)

#clean data
titleBasics_df = titleBasics_df.drop('originalTitle', 'runtimeMinutes').sort('tconst', ascending=True)
titleBasics_df.show(5, truncate=True)

+---------+---------+--------------------+-------+---------+-------+--------------------+
|   tconst|titleType|        primaryTitle|isAdult|startYear|endYear|              genres|
+---------+---------+--------------------+-------+---------+-------+--------------------+
|tt0000001|    short|          Carmencita|      0|     1894|     \N|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|      0|     1892|     \N|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      0|     1892|     \N|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|      0|     1892|     \N|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|      0|     1893|     \N|        Comedy,Short|
+---------+---------+--------------------+-------+---------+-------+--------------------+
only showing top 5 rows



In [3]:
#TITLE.AKAS
#tconst, ordering, title, region, language, types, attributes, isOriginalTitle
titleAkas_data = sc.textFile(filepath + '/IMDB/titleAkas.tsv')
titleAkas_rdd = titleAkas_data.filter(lambda l: 'ordering' not in l).map(lambda l: l.split(separator))

#create schema for df
taFields = []
taFields.append(StructField('titleId', StringType(), True))
taFields.append(StructField('ordering', StringType(), True))
taFields.append(StructField('title', StringType(), True))
taFields.append(StructField('region', StringType(), True))
taFields.append(StructField('language', StringType(), True))
taFields.append(StructField('types', StringType(), True))
taFields.append(StructField('attributes', StringType(), True))
taFields.append(StructField('isOriginalTitle', StringType(), True))                       

taSchema = StructType(taFields)

#create df
titleAkas_df= sqlContext.createDataFrame(titleAkas_rdd, taSchema)

#clean data
titleAkas_df = titleAkas_df.filter(titleAkas_df.region =='US')
titleAkas_df = titleAkas_df.dropDuplicates(['titleId']).sort('titleId', ascending=True)

titleAkas_df.show(5, truncate=True)

+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|  titleId|ordering|               title|region|language|      types|          attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|tt0000001|       3|          Carmencita|    US|      \N|         \N|                  \N|              0|
|tt0000002|       5|The Clown and His...|    US|      \N|         \N|literal English t...|              0|
|tt0000005|       1| Blacksmithing Scene|    US|      \N|alternative|                  \N|              0|
|tt0000006|       3|   Chinese Opium Den|    US|      \N|         \N|                  \N|              0|
|tt0000007|       1|Corbett and Court...|    US|      \N|         \N|                  \N|              0|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
only showing top 5 rows



In [9]:
#TITLE.PRINCIPALS
#tconst, ordering, nconst, category, job, characters
titlePrincipals_data = sc.textFile(filepath + '/IMDB/titlePrincipals.tsv')
titlePrincipals_rdd = titlePrincipals_data.filter(lambda l: 'ordering' not in l).map(lambda l: l.split(separator))

#create schema for df
tpFields = []
tpFields.append(StructField('tconst', StringType(), True))
tpFields.append(StructField('ordering', StringType(), True))
tpFields.append(StructField('nconst', StringType(), True))
tpFields.append(StructField('category', StringType(), True))
tpFields.append(StructField('job', StringType(), True))
tpFields.append(StructField('characters', StringType(), True))                     

tpSchema = StructType(tpFields)

#create df
titlePrincipals_df = sqlContext.createDataFrame(titlePrincipals_rdd, tpSchema)

#clean data
titlePrincipals_df = titlePrincipals_df.filter(titlePrincipals_df.category.contains("actor") | titlePrincipals_df.category.contains("actress"))
titlePrincipals_df = titlePrincipals_df.filter(titlePrincipals_df.characters !="\\N")
titlePrincipals_df = titlePrincipals_df.drop('job').sort('tconst', ascending=True)

titlePrincipals_df.show(2, truncate = True)

#write to csv file
titlePrincipals_df.repartition(1).write.option("sep", "|").format('com.databricks.spark.csv').save(outputFilepath + "/titlePrincipals_df.csv",header = 'true')

+---------+--------+---------+--------+--------------+
|   tconst|ordering|   nconst|category|    characters|
+---------+--------+---------+--------+--------------+
|tt0000005|       2|nm0653042|   actor| ["Assistant"]|
|tt0000005|       1|nm0443482|   actor|["Blacksmith"]|
+---------+--------+---------+--------+--------------+
only showing top 2 rows



In [4]:
#TITLE.RATINGS
#tconst, averageRating, numVotes
titleRatings_data = sc.textFile(filepath + '/IMDB/titleRatings.tsv')
titleRatings_rdd = titleRatings_data.filter(lambda l: 'averageRating' not in l).map(lambda l: l.split(separator))

#create schema for df
trFields = []
trFields.append(StructField('tconst', StringType(), True))
trFields.append(StructField('averageRating', StringType(), True))
trFields.append(StructField('numVotes', StringType(), True))                    

trSchema = StructType(trFields)

#create df
titleRatings_df= sqlContext.createDataFrame(titleRatings_rdd, trSchema)

titleRatings_df.show(2, truncate = True)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.8|    1416|
|tt0000002|          6.4|     167|
+---------+-------------+--------+
only showing top 2 rows



## Join title.Basics and title.Akas and title.Ratings

In [10]:
#Aliases
Basics = titleBasics_df.alias('Basics')
Akas = titleAkas_df.alias('Akas')
Ratings = titleRatings_df.alias('Ratings')

#join title.Basics to title.Akas
AkasBasics = Akas.join(Basics, Akas.titleId == Basics.tconst, how='left')

#clean data 1st join
AkasBasics = AkasBasics.filter(AkasBasics.isAdult =='0')
AkasBasics = AkasBasics.drop('ordering', 'language', 'types', 
                               'attributes', 'isOriginalTitle', 'tconst', 
                               'primaryTitle', 'isAdult')
AkasBasics = AkasBasics.filter(AkasBasics.titleType.contains("movie"))
AkasBasics = AkasBasics.filter(AkasBasics.startYear >='1950')

#join AkasBasics with title.Ratings
titleABR = AkasBasics.join(Ratings, AkasBasics.titleId == Ratings.tconst, how='left')

#clean data 2nd join
titleABR = titleABR.drop('tconst')

titleABR.show(3, truncate = True)

#write to csv file
titleABR.repartition(1).write.option("sep", "|").format('com.databricks.spark.csv').save(outputFilepath + "/titleABR.csv", header = 'true')

+---------+-----------------+------+---------+---------+-------+--------------------+-------------+--------+
|  titleId|            title|region|titleType|startYear|endYear|              genres|averageRating|numVotes|
+---------+-----------------+------+---------+---------+-------+--------------------+-------------+--------+
|tt0042899|   Ghost Mountain|    US|    movie|     1950|     \N|Action,Adventure,...|          6.8|     791|
|tt0042917|Salt Lake Raiders|    US|    movie|     1950|     \N|             Western|          6.5|      59|
|tt0043729|  Lawless Cowboys|    US|    movie|     1951|     \N|             Western|          6.1|       9|
+---------+-----------------+------+---------+---------+-------+--------------------+-------------+--------+
only showing top 3 rows



# BabyNames
## RDD Creation
### US

In [24]:
#US Baby Names
#NationalNames.csv
#Id,Name,Year,Gender,Count
nationalname_rdd_init = (sc.textFile(filepath + '/us-baby-names/NationalNames.csv'))
nationalname_rdd = nationalname_rdd_init.filter(lambda x: 'Gender' not in x)
nationalname_rdd = nationalname_rdd.map(lambda x: x.split(","))

### Per State

In [4]:
#State Baby Names
#StateNames.csv
#Id,Name,Year,Gender,State,Count
full_statename_rdd_init = (sc.textFile(filepath + '/us-baby-names/StateNames.csv'))
full_statename_rdd = full_statename_rdd_init.filter(lambda x: 'Gender' not in x).map(lambda x: x.split(","))

## US States

In [5]:
us_states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
us_states_abbreviations = list(us_states.keys())

# Analysis

## Top 10 movies for year x

In [22]:
separator = '|'
year = '2003'

#create rdd
titleABR_data = sc.textFile(filepath + '/IMDB_new/titleABR.csv/titleABR.csv')
titleABR_rdd = titleABR_data.filter(lambda l: 'region' not in l).map(lambda l: l.split(separator))

#get top 10 movies by year
top_movies_rdd = titleABR_rdd.filter(lambda x: x[4] == year).filter(lambda x: x[7] != '').filter(lambda x: ('Documentary') not in x[6]).filter(lambda x: float(x[8]) >99).sortBy(lambda x: float(x[7]), False)
print(top_movies_rdd.take(10))
top_movies_list = top_movies_rdd.map(lambda x: x[0]).take(10)


[['tt0167260', 'The Lord of the Rings: The Return of the King', 'US', 'movie', '2003', '\\N', 'Action,Adventure,Drama', '8.9', '1424076'], ['tt0332569', 'As a Bad Dream', 'US', 'movie', '2003', '\\N', 'Drama', '8.5', '225'], ['tt0346336', 'The Best of Youth', 'US', 'movie', '2003', '\\N', 'Drama,Romance', '8.5', '18505'], ['tt0339535', 'The Professional', 'US', 'movie', '2003', '\\N', 'Comedy,Drama', '8.4', '5163'], ['tt0347779', 'Pinjar', 'US', 'movie', '2003', '\\N', 'Drama', '8.1', '2262'], ['tt0353969', 'Memories of Murder', 'US', 'movie', '2003', '\\N', 'Action,Crime,Drama', '8.1', '93468'], ['tt0374546', 'Spring, Summer, Fall, Winter... and Spring', 'US', 'movie', '2003', '\\N', 'Drama,Romance', '8.1', '69526'], ['tt0266697', 'Kill Bill Part 1', 'US', 'movie', '2003', '\\N', 'Action,Crime,Thriller', '8.1', '865430'], ['tt0266543', 'Finding Nemo', 'US', 'movie', '2003', '\\N', 'Adventure,Animation,Comedy', '8.1', '829239'], ['tt0347304', 'Tomorrow May Never Come', 'US', 'movie', '

## Get principal characters for top 10

In [7]:
#create rdd
titlePrincipals_data = sc.textFile(filepath + '/IMDB_new/titlePrincipals_df.csv/titlePrincipals.csv')
titlePrincipals_rdd = titlePrincipals_data.filter(lambda l: 'category' not in l).map(lambda l: l.split(separator))

#filter out characters by id's in top 10 list
top_characters_rdd = titlePrincipals_rdd.filter(lambda x: x[0] in top_movies_list)                                             
top_characters_list = top_characters_rdd.map(lambda x: x[4]).collect()
for x in top_characters_list:
    print(x.replace('\"', '').replace('[', '').replace(']', '').replace('\\', ''))

Harry Potter
Hermione Granger
Ron Weasley
Professor Albus Dumbledore
Tommy Conlon
Paddy Conlon
Brendan Conlon
Tess Conlon
Skeeter Phelan
Aibileen Clark
Minny Jackson
Hilly Holbrook
Arjun Saluja
Imraan Qureshi
Kabir Dewan
Laila
Philippe
Driss
Yvonne
Magalie
Gustav,Constable
The Creature,Victor Frankenstein
The Creature,Victor Frankenstein
Female Creature
Gretel,Clarice
Nader
Simin
Razieh
Hojjat
Carl
Brenda
Phantom,Hans,Bob
Gatekeeper
Earl McGraw
Buck
Bill
Hattori Hanzo
Sofie Fatale
Vernita Green
Gogo Yubari
Budd
Harry S. Plinkett


## Popularity of x name by year

In [15]:
#name variable used for further analysis
name = "Trinity"

#isolating year variable with highest frequency
sortByCount = nationalname_rdd.filter(lambda x: x[1] == str(name)).sortBy(lambda x: int(x[4]), False)
year = sortByCount.first()
year = year[2]

#method definition for graph
def name_per_year_plot(name):
    print(name)
    pername_nationalname_rdd = nationalname_rdd.filter(lambda x: x[1] == str(name))
    # Create a trace

    trace = go.Scatter(
        x = pername_nationalname_rdd.map(lambda x: int(x[2])).collect(),
        y = pername_nationalname_rdd.map(lambda x: int(x[4])).collect()
    )

    data = [trace]
    
    return(py.iplot(data, filename='basic-line'))

#create graph for given name
name_per_year_plot(name)

Trinity


## Heatmap of popularity by state for most popular year

In [16]:
#NOTE: dataset US Baby Names contains only name instances where count/year >= 5
#connecting highest frequency year to popularity per state
name_frequency_state = full_statename_rdd.filter(lambda x: x[2] == year).filter(lambda x: x[1] == name)
states = name_frequency_state.map(lambda x: x[4]).collect()
amount_state = name_frequency_state.map(lambda x: x[5]).collect()

for x in range(0, len(states)):
    print(states[x] + '  ' + amount_state[x])

AK  22
AL  124
AR  82
AZ  160
CA  489
CO  99
CT  50
DC  11
DE  12
FL  350
GA  233
HI  34
IA  48
ID  35
IL  192
IN  154
KS  71
KY  80
KY  31
LA  139
LA  5
MA  67
MD  108
ME  29
MI  153
MN  58
MO  138
MS  74
MT  16
NC  202
ND  7
NE  31
NH  15
NJ  80
NM  49
NV  57
NY  204
OH  241
OK  130
OR  67
PA  181
RI  10
SC  107
SD  17
TN  147
TX  569
TX  7
UT  51
VA  162
VT  5
WA  104
WI  116
WV  40
WY  10


In [17]:
#us_states_abbreviations
#full_statename_rdd

trc = dict (
    type = 'choropleth',
    locations = states,
    locationmode = 'USA-states',
    colorscale ='YlOrRd',
    z = amount_state, 
    reversescale = True)
lyt = dict ( geo = dict (scope ='usa'))
map = go.Figure (data = [trc],
    layout = lyt)
py.iplot(map)

## Top 20 baby names for year x

In [33]:
#US Baby Names
#NationalNames.csv
#Id,Name,Year,Gender,Count

#F = female, M = male
gender = 'F'

#get sorted list of names by year
top_names_list = nationalname_rdd.filter(lambda x : x[2] == year).filter(lambda x: x[3] == gender).sortBy(lambda x: int(x[4]), False)

#show top 20
names = top_names_list.map(lambda x: x[1]).collect()
amount_name = top_names_list.map(lambda x: x[4]).collect()

for x in range(0, 20):
    print(names[x] + '\t\t' + amount_name[x])

Emily		25685
Emma		22701
Madison		20197
Hannah		17628
Olivia		16142
Abigail		15918
Alexis		14857
Ashley		14512
Elizabeth		14094
Samantha		13862
Isabella		13770
Sarah		13753
Grace		12770
Alyssa		12743
Lauren		11046
Kayla		10949
Brianna		10580
Jessica		10445
Taylor		10304
Sophia		9682
